Goal: Investigate different landmarks on LGI-PPGI

a. Raw Signal Extraction
- Skin extraction: convex hull
- 468 keypoints: patch, landmarks definition from thesis
    - Modified, should it also include the points inside and not just contour ? 
- 28 ROIs --> TODO how to combine them ? 
- Raw RGB Traces

b. HR Estimation 
- rPPG Methods: OMIT, POS, CHROM, LGI
- BVP Signal
- Estimated HR Values
- Evaluation: MAE, PCC + RMSE, SNR

In [1]:
%load_ext autoreload
%autoreload 2

import pyVHR as vhr
import numpy as np
from pyVHR.analysis.pipeline import Pipeline
from pyVHR.plot.visualize import *
import os
import plotly.express as px
from pyVHR.utils.errors import getErrors, printErrors, displayErrors

import constants
import pandas as pd

vhr.plot.VisualizeParams.renderer = 'vscode' 

In [ ]:
# -- LOAD A DATASET

dataset_name = 'lgi_ppgi'    
video_DIR, BVP_DIR = constants.get_dataset_paths(dataset_name)

dataset = vhr.datasets.datasetFactory(dataset_name, videodataDIR=video_DIR, BVPdataDIR=BVP_DIR)
allvideo = dataset.videoFilenames

videos = constants.get_video_settings(dataset_name)
print(videos)

# print the list of video names with the progressive index (idx)
for v in range(len(allvideo)):
  print(v, allvideo[v])

0 D:/datasets_rppg/MR-NIRP_indoor\Subject1_motion_940\Subject1_motion_940\RGB_corrected\Subject1_motion_940.avi
1 D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\RGB_corrected\Subject1_still_940.avi
2 D:/datasets_rppg/MR-NIRP_indoor\Subject2_motion_940\Subject2_motion_940\RGB_corrected\Subject2_motion_940.avi
3 D:/datasets_rppg/MR-NIRP_indoor\Subject2_still_940-002\Subject2_still_940\RGB_corrected\Subject2_still_940.avi
4 D:/datasets_rppg/MR-NIRP_indoor\Subject3_motion_940\Subject3_motion_940\RGB_corrected\Subject3_motion_940.avi
5 D:/datasets_rppg/MR-NIRP_indoor\Subject3_still_940-012\Subject3_still_940\RGB_corrected\Subject3_still_940.avi
6 D:/datasets_rppg/MR-NIRP_indoor\Subject4_motion_940\Subject4_motion_940\RGB_corrected\Subject4_motion_940.avi
7 D:/datasets_rppg/MR-NIRP_indoor\Subject4_still_940-004\Subject4_still_940\RGB_corrected\Subject4_still_940.avi
8 D:/datasets_rppg/MR-NIRP_indoor\Subject5_still_940-003\Subject5_still_940\RGB_corrected\Subject5_s

In [ ]:
# -- PARAMETER SETTING

wsize = 8        # seconds of video processed (with overlapping) for each estimate 
video_idx = 1    # index of the video to be processed
fname = dataset.getSigFilename(video_idx)
sigGT = dataset.readSigfile(fname)
bpmGT, timesGT = sigGT.getBPM(wsize)
videoFileName = dataset.getVideoFilename(video_idx)
print('Video processed name: ', videoFileName)
fps = vhr.extraction.get_fps(videoFileName)
print('Video frame rate:     ',fps)

Video processed name:  D:/datasets_rppg/MR-NIRP_indoor\Subject1_still_940-015\Subject1_still_940\RGB_corrected\Subject1_still_940.avi
Video frame rate:      30.0


# Visualize landmarks

In [ ]:
# Read lines in file
## TODO define this in extraction/utils.py
landmarks_file = 'landmarks.txt'
with open(landmarks_file, encoding='utf-8-sig') as f:
    lines = f.read().splitlines()

landmarks_list = dict()
for line in lines:
    landmarks_list[line.split(':')[0]] = [int(x) for x in line.split(':')[1][1:].replace(' ','').split(',')]

landamark_names = list(landmarks_list.keys())
print("Number of defined landmarks: ", len(landamark_names))

landmarks = landmarks_list[landamark_names[0]]

# -- VISUALIZE LANDMARK SUBSET
print('Num landmarks: ', len(landmarks))
vhr.plot.visualize_landmarks_list(landmarks_list=landmarks, image_file_name='../img/face.png')

Number of defined landmarks:  28
Num landmarks:  6


In [9]:
sig_extractor = vhr.extraction.SignalProcessing()
hol_sig = sig_extractor.extract_holistic(videoFileName)
# -- INTERACTIVE VISUALIZATION OF PATCHES
visualize_skin_coll = sig_extractor.get_visualize_skin()
print('Number of frames processed: ',len(visualize_skin_coll))
middle = int(len(visualize_skin_coll)/2)
vhr.plot.interactive_image_plot(visualize_skin_coll[middle:middle+3],1.0)

Number of frames processed:  0


In [6]:
sig_extractor = vhr.extraction.SignalProcessing()
sig_extractor.set_skin_extractor(vhr.extraction.SkinExtractionConvexHull('GPU'))

# set the number of seconds (0 for all video)
seconds = 0
sig_extractor.set_total_frames(seconds*fps)

# -- SET VISUALIZATION MODE 
sig_extractor.set_visualize_skin_and_landmarks(
      visualize_skin=True, 
      visualize_landmarks=True, 
      visualize_landmarks_number=True, 
      visualize_patch=True)

# landmarks = landmarks_list[landamark_names[0]]

ldmks_list = [x for x in list(vhr.extraction.CustomLandmarks().__dict__)]
landmarks = ['lower_medial_forehead', 'left_lower_lateral_forehead', 'right_lower_lateral_forehead', 'glabella','left_malar',
 'right_malar', 'left_lower_cheek', 'right_lower_cheek', 'chin',]
all_landmarks  = vhr.extraction.CustomLandmarks().get_all_landmarks()
landmarks = list(np.unique(sum([all_landmarks[ldmk] for ldmk in landmarks], [])))

# -- SET THE LANDMARK LIST
sig_extractor.set_landmarks(landmarks)

# -- PATCHES EXTRACTION
## TODO patches size ? 
sig_extractor.set_square_patches_side(60.0)
patch_sig = sig_extractor.extract_patches(videoFileName, "squares", "mean")
print('Size: (#frames, #landmarks, #channels) = ', patch_sig.shape)

Size: (#frames, #landmarks, #channels) =  (1815, 81, 3)


In [7]:
# -- INTERACTIVE VISUALIZATION OF PATCHES
visualize_patches_coll = sig_extractor.get_visualize_patches()
print('Number of frames processed: ',len(visualize_patches_coll))
middle = int(len(visualize_patches_coll)/2)
vhr.plot.interactive_image_plot(visualize_patches_coll[middle:middle+10],1.0)

Number of frames processed:  1815
Frame Shape (W,H) = ((640, 640))
None


interactive(children=(IntSlider(value=0, description='x', max=9), Output()), _dom_classes=('widget-interact',)…

# Test each landmark one by one

Take all 28 landmarks and compare them one by one

In [4]:
# test file just to make sure things work 
if os.path.exists(f"C:/Users/erolland/Documents/pyVHR/results/test_landmarks/h5/{dataset_name}_each_landmark_test.h5"):
    # delete file
    os.remove(f"C:/Users/erolland/Documents/pyVHR/results/test_landmarks/h5/{dataset_name}_each_landmark_test.h5")

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 

path_results = "../results/test_landmarks/" # general path for cfg
method = 'median'
dataset = dataset_name.upper()

# def RUN(dataset, method, filenameH5=None):
#   """Run methods on a dataset"""s

# set paths
print('## Dataset: ' + dataset)
print('Using...')
cd = os.getcwd() # current dir
# filenameH5 = path_results + 'h5/' + f"{dataset}_each_landmark.h5"
filenameH5 = path_results + f"{dataset}_each_landmark.h5"

print('   filename h5: ', filenameH5)
cfg = path_results  + 'cfg/'+ f"{dataset}_each_landmarks.cfg"
print('   cfg        :', cfg)
print('\n')

# pipeline
import pyVHR.analysis.pipelineLandmarks as custom_pipeline

pl = custom_pipeline.LandmarksPipeline()
res = pl.run_on_dataset(os.path.join(cd, cfg), verb=1)
res.saveResults(filenameH5)
print('Written file: ' + filenameH5 + '\n\n')

# RUN(dataset, 'median')

# print(res.dict)
# res.dataFrame

warnings.resetwarnings()

## Dataset: MR_NIRP
Using...
   filename h5:  ../results/test_landmarks/MR_NIRP_each_landmark.h5
   cfg        : ../results/test_landmarks/cfg/MR_NIRP_each_landmarks.cfg


** Run the test with the following config:
      dataset: MR_NIRP
      methods: ['CHROM', 'POS', 'LGI']
# CUDA devices:  2
# device number  0 :  NVIDIA GeForce RTX 3080
# device number  1 :  NVIDIA GeForce RTX 3080
 -  cuda device: True
 -  skin extractor: convexhull
Landmarks list :  [['lower_medial_forehead'], ['left_lower_lateral_forehead'], ['right_lower_lateral_forehead'], ['glabella'], ['upper_nasal_dorsum'], ['lower_nasal_dorsum'], ['soft_triangle'], ['left_ala'], ['right_ala'], ['nasal_tip'], ['left_lower_nasal_sidewall'], ['right_lower_nasal_sidewall'], ['left_mid_nasal_sidewall'], ['right_mid_nasal_sidewall'], ['philtrum'], ['left_upper_lip'], ['right_upper_lip'], ['left_nasolabial_fold'], ['right_nasolabial_fold'], ['left_temporal'], ['right_temporal'], ['left_malar'], ['right_malar'], ['left_lower_cheek'

c:\users\erolland\documents\pyvhr\pyVHR\utils\errors.py:218: RuntimeWarning: invalid value encountered in scalar divide
  snr = 10*np.log10(SPower/allPower)



    * Errors: RMSE = 4.42, MAE = 2.84, MAX = 13.25, PCC = 0.32, CCC = 0.26, SNR = nan
## method: LGI (patches) ['philtrum']


c:\users\erolland\documents\pyvhr\pyVHR\utils\errors.py:218: RuntimeWarning: invalid value encountered in scalar divide
  snr = 10*np.log10(SPower/allPower)



    * Errors: RMSE = 19.62, MAE = 9.54, MAX = 97.35, PCC = -0.31, CCC = -0.07, SNR = nan

## videoID: 10
D:/datasets_rppg/MR-NIRP_indoor\Subject6_still_940-005\Subject6_still_940\RGB_corrected\Subject6_still_940.avi
## method: CHROM (patches) ['philtrum']

    * Errors: RMSE = 23.27, MAE = 14.85, MAX = 61.39, PCC = 0.00, CCC = 0.00, SNR = nan
## method: POS (patches) ['philtrum']


...

c:\users\erolland\documents\pyvhr\pyVHR\utils\errors.py:218: RuntimeWarning: invalid value encountered in scalar divide
  snr = 10*np.log10(SPower/allPower)



    * Errors: RMSE = 8.60, MAE = 4.23, MAX = 43.19, PCC = 0.46, CCC = 0.38, SNR = nan
## method: LGI (patches) ['right_temporal']


c:\users\erolland\documents\pyvhr\pyVHR\utils\errors.py:218: RuntimeWarning: invalid value encountered in scalar divide
  snr = 10*np.log10(SPower/allPower)



    * Errors: RMSE = 3.17, MAE = 1.85, MAX = 14.78, PCC = 0.81, CCC = 0.81, SNR = nan

## videoID: 13
D:/datasets_rppg/MR-NIRP_indoor\Subject8_motion_940\Subject8_motion_940\RGB_corrected\Subject8_motion_940.avi
## method: CHROM (patches) ['right_temporal']

    * Errors: RMSE = 18.98, MAE = 12.96, MAX = 71.11, PCC = 0.38, CCC = 0.20, SNR = -2.79
## method: POS (patches) ['right_temporal']

    * Errors: RMSE = 23.85, MAE = 19.04, MAX = 74.22, PCC = -0.06, CCC = -0.02, SNR = -3.81
## method: LGI (patches) ['right_temporal']

    * Errors: RMSE = 19.28, MAE = 13.84, MAX = 62.35, PCC = 0.34, CCC = 0.18, SNR = -2.77

## videoID: 14
D:/datasets_rppg/MR-NIRP_indoor\Subject8_still_940-001\Subject8_still_940\RGB_corrected\Subject8_still_940.avi
## method: CHROM (patches) ['right_temporal']

    * Errors: RMSE = 23.68, MAE = 14.38, MAX = 82.88, PCC = 0.16, CCC = 0.04, SNR = -2.10
## method: POS (patches) ['right_temporal']

    * Errors: RMSE = 38.10, MAE = 27.25, MAX = 123.25, PCC = 0.18, CC

c:\Users\erolland\.conda\envs\project\lib\site-packages\tables\path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'self.dataFrame'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [9]:
x = pd.read_hdf(filenameH5)
print(x.shape)
x.head(1)

(1260, 19)


,method,dataset,videoIdx,sigFilename,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,bpmGT,bpmES,bpmES_mad,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,CHROM,MR_NIRP,0,,D:/datasets_rppg/MR-NIRP_indoor\Subject1_motio...,[1.9098900887358645],[1.0337082435344827],[0.8147077415848941],[0.7991721563067217],[0.5757400806559699],[8.640625],"[1.318359375, 0.439453125, 0.87890625, 0.87890...","[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[72.509765625, 75.5859375, 77.34375, 78.222656...",,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",51.750714,[lower_medial_forehead]


In [12]:
rois = constants.get_rois()

print("Reading file: ", filenameH5)
df_all = pd.read_hdf(filenameH5)
df_all = df_all.drop(columns=['sigFilename', 'bpmES_mad'])
df_all['videoFilename'] = df_all['videoFilename'].apply(lambda x: x.split('\\')[2])
df_all['dataset'] = df_all['dataset'].str.lower() ## check

# add DTW
from fastdtw import fastdtw
df_all.insert(11, 'DTW', None)
for row in df_all.itertuples():
  distance, path = fastdtw(row.bpmGT, row.bpmES)
  df_all.loc[row.Index, 'DTW'] = [distance]

# add landmarks 
for roi in list(rois.keys()):
    df_all.loc[df_all['landmarks'].apply(lambda x: x[0]).isin(rois[f'{roi}']),'ROI'] = roi
df_all['landmarks'] = df_all['landmarks'].apply(lambda x: tuple(x))

# save
df_all.to_hdf(f'../results/test_landmarks/h5/{dataset_name}/{dataset_name}_each_landmark.h5', key='df')

print(df_all.landmarks.unique().size, df_all.videoFilename.unique().size)
print(df_all.ROI.unique())
df_all.head(1)

Reading file:  ../results/test_landmarks/MR_NIRP_each_landmark.h5
28 15


,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,DTW,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,CHROM,mr_nirp_forehead,0,Subject1_motion_940,[1.9098900887358645],[1.0337082435344827],[0.8147077415848941],[0.7991721563067217],[0.5757400806559699],[8.640625],"[1.318359375, 0.439453125, 0.87890625, 0.87890...",[63.208984375],"[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[72.509765625, 75.5859375, 77.34375, 78.222656...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",51.750714,"(lower_medial_forehead,)"


In [13]:
df_all.to_hdf(filenameH5, key='df', mode='w')

C:\Users\erolland\AppData\Local\Temp\3\ipykernel_24020\1393275649.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['method', 'dataset', 'videoFilename', 'RMSE', 'MAE', 'PCC', 'CCC',
       'SNR', 'MAX', 'MAD', 'DTW', 'bpmGT', 'bpmES', 'timeGT', 'timeES',
       'landmarks'],
      dtype='object')]

  df_all.to_hdf(filenameH5, key='df', mode='w')


In [14]:
df_all = pd.read_hdf(filenameH5)
df_all.head(2)

,method,dataset,videoIdx,videoFilename,RMSE,MAE,PCC,CCC,SNR,MAX,MAD,DTW,bpmGT,bpmES,timeGT,timeES,TIME_REQUIREMENT,landmarks
0,CHROM,mr_nirp_forehead,0,Subject1_motion_940,[1.9098900887358645],[1.0337082435344827],[0.8147077415848941],[0.7991721563067217],[0.5757400806559699],[8.640625],"[1.318359375, 0.439453125, 0.87890625, 0.87890...",[63.208984375],"[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[72.509765625, 75.5859375, 77.34375, 78.222656...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",51.750714,"(lower_medial_forehead,)"
1,POS,mr_nirp_forehead,0,Subject1_motion_940,[2.0058221752407497],[1.1761516702586208],[0.8074041855861901],[0.7917540665403351],[0.27704660170551004],[9.080078125],"[2.197265625, 0.439453125, 0.439453125, 0.8789...",[71.1328125],"[72.0, 71.0, 67.5, 67.0, 70.0, 75.5, 78.0, 78....","[68.994140625, 76.46484375, 77.783203125, 78.2...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12....",71.463084,"(lower_medial_forehead,)"
